In [1]:
import pandas as pd
import numpy as np

In [2]:
import scipy as sc
from statsmodels.stats.weightstats import *
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import chi2_contingency

In [14]:
from statsmodels.sandbox.stats.multicomp import multipletests 
import itertools
from statsmodels.stats.descriptivestats import sign_test 
from statsmodels.stats.weightstats import zconfint
from scipy.stats import wilcoxon
from scipy import stats

In [4]:
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

C:\Users\gruyuri\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [5]:
ds = pd.read_csv('gene_high_throughput_sequencing.csv', sep = ',', header = 0)
ds.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [6]:
ds.shape

(72, 15750)

In [7]:
ds.Diagnosis.unique()

array(['normal', 'early neoplasia', 'cancer'], dtype=object)

In [9]:
ds_normal = ds[ds.Diagnosis == 'normal']
ds_early = ds[ds.Diagnosis == 'early neoplasia']
ds_cancer = ds[ds.Diagnosis == 'cancer']


,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
24,STT5424_Breast_001_EN,early neoplasia,2.516305,11.430887,18.506550,13.969049,20.957007,10.374992,8.414330,68.513944,...,4.488498,1.314098,1.314098,1.314098,3.307073,1.314098,25.059902,1.314098,1.314098,1.314098
25,STT5431_Breast_002_EN,early neoplasia,1.937270,9.686352,23.541357,15.295034,18.815807,11.128772,9.155183,67.951908,...,3.709591,1.937270,1.937270,1.937270,1.937270,1.937270,26.349727,1.937270,1.937270,1.937270
26,STT5442_Breast_004_EN,early neoplasia,1.405858,15.119783,17.985461,17.237294,21.824785,4.801919,5.796501,67.064975,...,7.029290,1.405858,1.405858,1.405858,1.405858,1.405858,29.254009,1.405858,1.405858,1.405858
27,STT5445_Breast_005_EN,early neoplasia,2.131757,8.789458,12.731187,6.395270,19.185811,14.922297,4.082003,52.028259,...,9.453726,2.131757,2.131757,2.131757,2.131757,2.131757,23.189490,2.131757,2.131757,2.131757
28,STT5511_Breast_011_EN,early neoplasia,2.421766,7.830416,18.283935,15.229320,29.234970,15.779540,8.098199,50.224123,...,5.976863,1.264726,1.264726,1.264726,1.264726,1.264726,19.849890,1.264726,1.264726,1.264726


In [12]:
# ds_early.shape # 25, 15750
# ds_cancer.shape # 23, 15750

(23, 15750)

In [15]:
p_values = pd.DataFrame(index = [0, 1], columns = ds.columns[2:])
for column in p_values.columns:
    p_values[column][0] = stats.ttest_ind(ds[ds['Diagnosis'] == 'normal'][column], ds[ds['Diagnosis'] == 'early neoplasia'][column], equal_var = False).pvalue
    p_values[column][1] = stats.ttest_ind(ds[ds['Diagnosis'] == 'early neoplasia'][column], ds[ds['Diagnosis'] == 'cancer'][column], equal_var = False).pvalue


In [16]:
t1 = (p_values.iloc[0] < 0.05).sum()
print t1

1575


In [17]:
t2 = (p_values.iloc[1] < 0.05).sum()
print t2

3490


In [18]:
import statsmodels.stats.multitest as smm

In [59]:
reject_0, p_corrected_0 = smm.multipletests(p_values.iloc[0], alpha=0.025, method='holm')[:2]
reject_1, p_corrected_1 = smm.multipletests(p_values.iloc[1], alpha=0.025, method='holm')[:2]

In [32]:
ds1_temp = pd.DataFrame()
ds1_temp['reject'] = reject_0
ds1_temp['p_corrected'] = p_corrected_0
ds1_temp['p_values'] = p_values.as_matrix()[0,:]
ds1_temp.head()

,reject,p_corrected,p_values
0,False,1,0.690766
1,False,0.500174,3.17853e-05
2,False,1,0.0602727
3,False,1,0.826429
4,False,1,0.0498762


In [20]:
def fold_change(C, T):
    if T > C:
        r = T/float(C)
    else:
        r = -C/float(T)
    return r


In [33]:
ds2_temp = pd.DataFrame()
ds2_temp['p_corrected'] = ds1_temp[(ds1_temp.p_corrected < 0.025) & (ds1_temp.reject == True)]['p_corrected']
ds2_temp['p_values'] = ds1_temp[(ds1_temp.p_corrected < 0.025) & (ds1_temp.reject == True)]['p_values']
ds2_temp.head()

,p_corrected,p_values
7244,0.0125274,7.95543e-07
9820,0.00133838,8.49874e-08


In [34]:
fct1 = []
for index, row in ds2_temp.iterrows():
    fct1.append( fold_change(row['p_values'], row['p_corrected']) )
    
c1 = 0
for item in fct1:
    if abs(item) > 1.5:
        c1 = c1 + 1
        
print c1

2


In [60]:
ds1_temp = pd.DataFrame()
ds1_temp['reject'] = reject_1
ds1_temp['p_corrected'] = p_corrected_1
ds1_temp['p_values'] = p_values.as_matrix()[1,:]
ds1_temp.head()

,reject,p_corrected,p_values
0,False,1,0.413735
1,False,1,0.653429
2,False,1,0.0795556
3,False,1,0.287581
4,False,1,0.463292


In [61]:
ds2_temp = pd.DataFrame()
ds2_temp['p_corrected'] = ds1_temp[(ds1_temp.p_corrected < 0.025) & (ds1_temp.reject == True)]['p_corrected']
ds2_temp['p_values'] = ds1_temp[(ds1_temp.p_corrected < 0.025) & (ds1_temp.reject == True)]['p_values']

,p_corrected,p_values
47,0.00357521,2.27604e-07
283,0.00170862,1.08705e-07
316,0.0157077,1.00209e-06
1105,0.00163741,1.04168e-07
1238,0.0172922,1.10331e-06
1309,0.00643903,4.10208e-07
1667,0.00112366,7.14617e-08
2033,0.000118821,7.55042e-09
2288,2.20269e-06,1.3988e-10
2289,1.17196e-06,7.44199e-11


In [62]:
c1 = 0
for num, row in ds2_temp.iterrows():
    c = ds[ds['Diagnosis'] == 'early neoplasia'].iloc[:, num + 2].mean()
    t = ds[ds['Diagnosis'] == 'cancer'].iloc[:, num + 2].mean()
    fct1 = fold_change (c, t)
    if abs(fct1) > 1.5:
        c1 = c1 + 1
print c1

77


In [38]:
fct1 = []
for index, row in ds2_temp.iterrows():
    fct1.append( fold_change(row['p_values'], row['p_corrected']) )
    
c1 = 0
for item in fct1:
    if abs(item) > 1.5:
        c1 = c1 + 1
        
print c1

79


In [39]:
reject_0, p_corrected_0 = smm.multipletests(p_values.iloc[0], alpha=0.025, method='fdr_bh')[:2]
reject_1, p_corrected_1 = smm.multipletests(p_values.iloc[1], alpha=0.025, method='fdr_bh')[:2]

In [46]:
ds1_temp = pd.DataFrame()
ds1_temp['reject'] = reject_0
ds1_temp['p_corrected'] = p_corrected_0
ds1_temp['p_values'] = p_values.as_matrix()[0,:]
ds1_temp.head()

,reject,p_corrected,p_values
0,False,0.966511,0.690766
1,False,0.0356976,3.17853e-05
2,False,0.536103,0.0602727
3,False,0.980777,0.826429
4,False,0.499016,0.0498762


In [47]:
ds2_temp = pd.DataFrame()
ds2_temp['p_corrected'] = ds1_temp[(ds1_temp.p_corrected < 0.025) & (ds1_temp.reject == True)]['p_corrected']
ds2_temp['p_values'] = ds1_temp[(ds1_temp.p_corrected < 0.025) & (ds1_temp.reject == True)]['p_values']
ds2_temp.head()

,p_corrected,p_values
4627,0.00874891,2.22223e-06
7244,0.00626411,7.95543e-07
8250,0.00874891,2.01547e-06
9820,0.00133838,8.49874e-08


In [42]:
fct1 = []
for index, row in ds2_temp.iterrows():
    fct1.append( fold_change(row['p_values'], row['p_corrected']) )
    
c1 = 0
for item in fct1:
    if abs(item) > 1.5:
        c1 = c1 + 1
        
print c1

4


In [56]:
ds1_temp = pd.DataFrame()
ds1_temp['reject'] = reject_1
ds1_temp['p_corrected'] = p_corrected_1
ds1_temp['p_values'] = p_values.as_matrix()[1,:]
ds1_temp.head()

,reject,p_corrected,p_values
0,False,0.675195,0.413735
1,False,0.836406,0.653429
2,False,0.288873,0.0795556
3,False,0.563007,0.287581
4,False,0.712214,0.463292


In [57]:
ds2_temp = pd.DataFrame()
ds2_temp['p_corrected'] = ds1_temp[(ds1_temp.p_corrected < 0.025) & (ds1_temp.reject == True)]['p_corrected']
ds2_temp['p_values'] = ds1_temp[(ds1_temp.p_corrected < 0.025) & (ds1_temp.reject == True)]['p_values']
ds2_temp.head()

,p_corrected,p_values
8,0.0175562,0.00074024
17,0.0094874,0.000294599
44,0.022992,0.00115486
47,8.61452e-05,2.27604e-07
54,0.0151983,0.000597396


In [58]:
c1 = 0
for num, row in ds2_temp.iterrows():
    c = ds[ds['Diagnosis'] == 'early neoplasia'].iloc[:, num + 2].mean()
    t = ds[ds['Diagnosis'] == 'cancer'].iloc[:, num + 2].mean()
    fct1 = fold_change (c, t)
    if abs(fct1) > 1.5:
        c1 = c1 + 1
print c1

524
